# pystan for subMM SEDs

Notebook to use the [`stan`](http://mc-stan.org) sampler (with the [`pystan`](https://pystan.readthedocs.org/en/latest/) interface) to sample submm SED models.

### Issues:

* Can I use my getdist (or A Lewis') with this output?
  * Yes: see `pystan2getdist`
* how do I deal with the mixture model with "duplicate" parameters?
  * currently using `positive ordered` vector of vector of temperatures
  * seems to work *if* you add an explicit uniform(3,100) prior
* Add optical-depth model -- *done*, but perhaps problems with parameterization?
  * Yes: scale to fiducial $\nu_0$, but this means likelihood is const. as $\nu_0\to\infty$
  * So require prior on $\nu_0$, currently `exponential()` in $\nu_0/1000$
  
### To do
* hook up to SED plotting code
  * Done -- required modifying pythoc code optical depth model
  * can do more within pystan?
* add $\chi^2$ as a parameter? (add to `generated quantities`)

In [1]:
from __future__ import print_function

import os
import sys
import warnings

## python 2/3 compatibility
try:
    import cPickle as pickle
except ImportError:
    import pickle
import gzip
try:
    from importlib import reload
except ImportError:
    pass

from six import iteritems

import matplotlib.pyplot as plt
import numpy as np
import pystan
import getdist
import getdist.plots

# sys.path.append(os.environ['HOME']+'/home/proj/stats/MCMC')

#### code from my MCMC sampler
from MCMC.submmSED import data      ## for reading data
from MCMC.submmSED import model     ## needed for plotting SED models (duplicates some STAN code)

from MCMC.submmSED import pystan_utils
from MCMC.submmSED import pystan_submm

%matplotlib inline

do_pickle = False
reload(getdist)
reload(getdist.plots)
reload(pystan_utils)
reload(pystan_submm)
reload(pystan_submm.model)

min Temp = 3.000000 K; max Temp = 100.000000 K
min Temp = 3.000000 K; max Temp = 100.000000 K


<module 'MCMC.submmSED.model' from '/Users/jaffe/home/proj/stats/MCMC/MCMC/submmSED/model.pyc'>

## Read data

In [2]:
import pkg_resources 
herus_file = pkg_resources.resource_filename("MCMC.submmSED", "dat/august6table.csv")
reload(data)
d = data.readfluxes_DLC_2014(filename=herus_file)

## convert to dictionary
dat = dict((dobj.name, dobj) for dobj in d)

## STAN code

In [3]:
submm_data = """
    data {

        int<lower=1> N_comp; // number of greybody components (fixed model parameter)

        int<lower=1> N_band;   // number of photometric bands
        vector[N_band] nu_obs; // observed frequency
        vector[N_band] flux;   // observed flux
        vector[N_band] sigma;  // error
        real z;                // redshift

    }
    
    transformed data {
        vector[N_band] nu;     // rest frame frequency
        nu = (1+z)*nu_obs;    // could do this outside of stan...
    }
    
"""

submm_functions = """
    functions {
        
        real greybody(real beta, real T, real nu) {
          // greybody, normalized to unit flux at nu=nu_0
            real h_over_k = 0.04799237;       //  K/Ghz
            real nu_bar = 1000;
            
            real x = h_over_k * nu / T;
            real x_bar = h_over_k * nu_bar / T;
            
            return (pow(nu/nu_bar, 3+beta) * expm1(x_bar) / expm1(x));
        }
    }
"""

submm_model_greybody_1 = """
    // single-component greybody -- not used
    parameters {
        real amplitude<lower=0.0>;
        real T<lower=3, upper=100.0>; // Temperature in K
        real beta<lower=0, upper=3>;  // greybody factor
    }
    
    transformed parameters {
        real totalflux<lower=0>[N_band];
        for (band in 1:N_band)
            totalflux[band] = amplitude * greybody(beta, T, nu[band])
    }
    
    model {
        // no explicit priors on parameters -- improper... 
        flux ~ normal(totalflux, sigma)
    }
    
"""

submm_model_greybody = """
    parameters {
    // nb. N_comp, N_band are data
//        vector[N_comp] log10amplitude;
        vector<lower=0>[N_comp] amplitude;
        positive_ordered[N_comp] T;
        vector<lower=0, upper=3>[N_comp] beta;  // greybody factor    
    }
    
// not used, since this is only needed if you want to keep track of these parameters.
//    transformed parameters {
//        real<lower=0> fluxes[N_band, N_comp];
//        real<lower=0> totalflux[N_band];
//        real<lower=0> amplitude[N_comp];
//        
//        for (comp in 1:N_comp)
//            amplitude[comp] = pow(10.0, log10amplitude[comp]);
//        
//        for (band in 1:N_band) {
//            for (comp in 1:N_comp) {  // vectorize over this?
//                fluxes[band, comp] = amplitude[comp] * greybody(beta[comp], T[comp], nu[band]);
//            }
//           totalflux[band] = sum(fluxes[band]);
//        }
//    }
    
    model {
        real fluxes[N_band, N_comp];
        vector[N_band] totalflux;
        
        for (band in 1:N_band) {
            for (comp in 1:N_comp) {  // vectorize over this?
                fluxes[band, comp] = amplitude[comp] * greybody(beta[comp], T[comp], nu[band]);
            }
            totalflux[band] = sum(fluxes[band]);
        }

        // try a proper prior on temperature; needed since ordered vectors don't have limits
        T ~ uniform(3,100);
        flux ~ normal(totalflux, sigma);
    }
    
"""


submm_model_greybody_b2 = """

//// like submm_model_greybody, but with fixed beta=2 
    parameters {
    // nb. N_comp, N_band are data
        vector<lower=0>[N_comp] amplitude;
        positive_ordered[N_comp] T;
    }

    model {
        real fluxes[N_band, N_comp];
        vector[N_band] totalflux;

        for (band in 1:N_band) {
            for (comp in 1:N_comp) {
                fluxes[band, comp] = amplitude[comp] * greybody(2.0, T[comp], nu[band]);
            }
            totalflux[band] = sum(fluxes[band]);
        }

        // try a proper prior on temperature; needed since ordered vectors don't have limits
        T ~ uniform(3,100);
        flux ~ normal(totalflux, sigma);
    }
    
"""

submm_model_optically_thick = """
    // model a submm SED as an optically-thick black body: flux = A (1-exp(-tau)) B_nu(T);  tau = (nu/nu0)^b
    // for consistency, allow varying N_comp, but really fix N_comp=1
    // since we typically have nu0~1000, scale that factor out to keep it O(1)
    // normalize by (1-exp(-taubar)) -- this removes the degeneracy with amplitude, **but** means that
    //      the likelihood is indep. of nu0 for nu0>>max(nu, nubar), so also give a prior on nu0.
    parameters {
    // nb. N_comp, N_band are data
        vector<lower=0.0>[N_comp] amplitude;
        positive_ordered[N_comp] T;
        vector<lower=0.001, upper=1000.0>[N_comp] nu0;
        vector<lower=0.0,upper=3.0>[N_comp] beta;
    }
    
    model {
        real fluxes[N_band, N_comp];
        vector[N_band] totalflux;
        real tau;
        real tau_bar;
        real nu0_norm;        
        real nu_bar = 1000;  // to remove degen. with amplitude. Should be as in the greybody definition.

        for (band in 1:N_band) {
            for (comp in 1:N_comp) {
                nu0_norm = 1000.0*nu0[comp];
                tau = pow(nu[band]/nu0_norm, beta[comp]);
                tau_bar = pow(nu_bar/nu0_norm, beta[comp]);
                fluxes[band, comp] = amplitude[comp] *(expm1(-tau)/expm1(-tau_bar)) * greybody(0.0, T[comp], nu[band]);
            }
            totalflux[band] = sum(fluxes[band]);
        }

        // try a proper prior on temperature; needed since ordered vectors don't have limits
        T ~ uniform(3,100);
//        nu0 ~ normal(1,3);
        nu0 ~ exponential(1.0/3.0);
        flux ~ normal(totalflux, sigma);
    }
    
"""

In [4]:
pickle_models=True
get_pickled_models = False

greybody_code = submm_functions + submm_data + submm_model_greybody
greybody_b2_code = submm_functions + submm_data + submm_model_greybody_b2
greybody_thick_code = submm_functions + submm_data + submm_model_optically_thick

if get_pickled_models:
    with gzip.open("greybody_model_stan.pkl.gz", "rb") as f:
        greybody_model = pickle.load(f)
    with gzip.open("greybody_b2_model_stan.pkl.gz", "rb") as f:
        greybody_b2_model = pickle.load(f)
    with gzip.open("greybody_thick_model_stan.pkl.gz", "rb") as f:
        greybody_thick_model = pickle.load(f)
else:
    greybody_model = pystan.StanModel(model_code=greybody_code, model_name="greybody")
    greybody_b2_model = pystan.StanModel(model_code=greybody_b2_code, model_name="greybody_beta2")
    greybody_thick_model = pystan.StanModel(model_code=greybody_thick_code, model_name="greybody_thick")

if pickle_models:
    with gzip.open("greybody_model_stan.pkl.gz", "wb") as f:
        pickle.dump(greybody_model, f)
    with gzip.open("greybody_b2_model_stan.pkl.gz", "wb") as f:
        pickle.dump(greybody_b2_model, f)
    with gzip.open("greybody_thick_model_stan.pkl.gz", "wb") as f:
        pickle.dump(greybody_thick_model, f)



INFO:pystan:COMPILING THE C++ CODE FOR MODEL greybody_0fad9fc64c92260ba395a2e1131309d2 NOW.


KeyboardInterrupt: 

In [ ]:
def pystan_process(dat, models=range(5), niter=4000, nchains=4):
    """
    process my submm MCMC data with pystan. Requires pystan code currently defined and compiled in the python notebook
    """

## the try/except are for catching printing problems that 
##     manifest when an unbounded parameter wanders off to infinity (fixed by T~uniform(3,100))
### order is 1comp, 2comp, 1compb2, 2compb2, thick -- only do those models in the iterable "models"

    allfits = {}
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore')
        for name, dobj in iteritems(dat):
            imodel=0
            assert name==dobj.name
            print("============="+dobj.name+"=============")
            allfits[dobj.name] = []
            for n_comp in (1,2):
                if imodel in models:
                    fit_beta = greybody_model.sampling(data=pystan_utils.makeStanDict(dobj, N_comp=n_comp),
                                             iter=niter, chains=nchains)
                    allfits[dobj.name].append(fit_beta)
                    try:
                        sfit=str(fit_beta)
                        print(sfit[:sfit.find("Samples were drawn")-1])
                    except OverflowError:
                        print("***** Can't print! %s %d unconstrained beta ******" % (dobj.name, n_comp))
                        print(fit_beta.summary())
                imodel += 1

            for n_comp in (1,2):
                if imodel in models:
                    fit_b2 = greybody_b2_model.sampling(data=pystan_utils.makeStanDict(dobj, N_comp=n_comp),
                                                 iter=niter, chains=nchains)
                    allfits[dobj.name].append(fit_b2)
                    try:
                        sfit=str(fit_b2)
                        print(sfit[:sfit.find("Samples were drawn")-1])
                    except OverflowError:
                        print("***** Can't print! %s %d beta=2 ******"  % (dobj.name, n_comp))
                        print(fit_b2.summary())
                imodel += 1

            if imodel in models:
                fit_thick = greybody_thick_model.sampling(data=pystan_utils.makeStanDict(dobj, N_comp=1),
                                                iter=niter, chains=nchains)
                allfits[dobj.name].append(fit_thick)
                try:
                    sfit=str(fit_thick)
                    print(sfit[:sfit.find("Samples were drawn")-1])
                except OverflowError:
                    print("***** Can't print! %s %d optically thick ******" % (dobj.name, 1))
                    print(fit_thick.summary())
            imodel += 1
    return allfits

## test on small number of objects and model

In [ ]:
allfits = {}
# IRAS14348-1447 IRAS20087-0308 IRAS20414-1651 IRAS00188-0856 IRAS09022-3615

iter = 4000
chains = 4

for objname in ("IRAS14348-1447", "IRAS20087-0308", "IRAS20414-1651", "IRAS00188-0856", "IRAS09022-3615"):
    print(objname)
    assert dat[objname].name == objname

    with warnings.catch_warnings():
        warnings.filterwarnings('ignore')
        fit_1comp = greybody_model.sampling(data=pystan_utils.makeStanDict(dat[objname], N_comp=1),
                      iter=iter, chains=chains)
        fit_2comp = greybody_model.sampling(data=pystan_utils.makeStanDict(dat[objname], N_comp=2),
                      iter=iter, chains=chains)

        fit_1comp_b2 = greybody_b2_model.sampling(data=pystan_utils.makeStanDict(dat[objname], N_comp=1),
                      iter=iter, chains=chains)
        fit_2comp_b2 = greybody_b2_model.sampling(data=pystan_utils.makeStanDict(dat[objname], N_comp=2),
                      iter=iter, chains=chains)

        fit_thick = greybody_thick_model.sampling(data=pystan_utils.makeStanDict(dat[objname], N_comp=1),
                      iter=4000, chains=4)
    
    allfits[objname]=(fit_1comp, fit_2comp, fit_1comp_b2, fit_2comp_b2, fit_thick) 

In [ ]:
for objname, fits in iteritems(allfits):
    print(objname)
    for fit in fits:
#         print(fit.extract(permuted=True).keys())
#         print(fit.extract(permuted=False).shape)
#         print(fit.model_pars)
#         print(fit.extract(permuted=True)['amplitude'].shape)
        # print([a.shape for a in fit.extract(permuted=True)['amplitude'].T])
        sfit=str(fit)
        print(sfit[:sfit.find("Samples were drawn")-1])  ## delete some useless verbiage
        fit.plot()
        gd1 = pystan_utils.pystan2getdist(fit)
        g = getdist.plots.getSubplotPlotter()
        g.triangle_plot(gd1)

In [ ]:
gd1 = pystan_utils.pystan2getdist(fit)
g=getdist.plots.getSubplotPlotter()

In [ ]:
labs = ("1 comp", "2 comp", r"1 $\beta=2$", r"2 $\beta=2$", "thick")
for objname, fits in iteritems(allfits):
    first = True
    for fit, lab in zip(fits, labs):
        pystan_submm.plot_pystan(dat[objname], fit, label=lab, first=first)
        first = False
    plt.title(objname)
    plt.legend(loc='best')
    plt.figure()

## Loop over all models and objects -- Dave & Michelle data

In [ ]:
allfits = pystan_process(dat)
# allfits = pystan_process(dat1)

### Postprocessing

In [ ]:
if do_pickle:
    with gzip.open("greybody_fits_stan.pkl.gz", "wb") as f:
        pickle.dump(allfits, f)

In [ ]:
# pystan_utils.pystan_postprocess_text(allfits, "pystan_greybody_DLC.out")
pystan_utils.pystan_postprocess_text(allfits, "pystan_greybody_DLC.out")

In [ ]:
gds = pystan_utils.pystan_postprocess_togetdist(allfits, texdict=pystan_utils.texdict, do_pickle=do_pickle)
# with gzip.open("greybody_getdist.pkl.gz", "rb") as f:
#     gds = pickle.load(f)

In [ ]:
pystan_utils.pystan_postprocess_getdist_plot(gds)

In [ ]:
pystan_submm.pystan_postprocess_SED_plot(allfits, dat)

In [ ]:
dat = dat1
pystan_utils.pystan_postprocess_csv(allfits, dat, label="DLC1_")

## all objects -- DLC Dec 2015 data

In [ ]:
files = ["./dat/herus_phot.dat",  "./dat/magdis_ulirgs.dat"]
d = []
for f in files:
    d.extend(data.readfluxes_DLC_2014(filename=f, getArp220=False))

## convert to dictionary
dat = dict((dobj.name, dobj) for dobj in d)

In [ ]:
allfits_1215 = pystan_process(dat, models=[0,], niter=100000)
if do_pickle:
    with gzip.open("greybody_fits_stan_DLC_1215.pkl.gz", "wb") as f:
        pickle.dump(allfits_1215, f)

In [ ]:
gds = pystan_utils.pystan_postprocess_togetdist(allfits_1215)
pystan_utils.pystan_postprocess_getdist_plot(gds, label="DLC_1215")

In [ ]:
pystan_submm.pystan_postprocess_SED_plot(allfits_1215, dat, label="DLC_1215")